In [1]:
# for some reason I have to do this manually because the import clip_main
# line doesn't work
import copy
import os
import uuid

import torch
import torch as ch
import torch.nn as nn
from fastargs import Param, Section
from fastargs.validation import And, OneOf
import numpy as np
import src.config_parse_utils as config_parse_utils
from src.eval_utils import evaluate_model
from src.trainer import LightWeightTrainer
from src.models_and_optimizers import create_clip_model, load_model
import src.dist_utils as dist_utils
import src.data_utils as data_utils
from transformers import EsmTokenizer
import src.loader as loaders_utils
import sys
import webdataset as wds
import tqdm
import tensorflow as tf
import os
import logging
from functools import partial
import src.loader as loader_utils
import src.zipdataset as zipdataset_utils
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL
logging.getLogger('tensorflow').setLevel(logging.FATAL)

/home/gridsan/lguan/miniconda3/envs/rla/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-16 15:21:35.820481: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-16 15:21:50.960465: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-16 15:22:31.132618: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: C

In [2]:
import src.models_and_optimizers as model_utils
from types import SimpleNamespace
from clip_main import get_wds_loaders
from transformers import EsmTokenizer
import src.data_utils as data_utils
import os
import torch
from tqdm import tqdm
from torch.cuda.amp import autocast
from transformers import EsmTokenizer, EsmModel
import esm as esmlib

In [3]:
## GENERAL SETUP (CHANGE PATHS AS NEEDED)
ROOT = "/home/gridsan/lguan/keating/rla/model_weights"
model_dir = "version_0/" 
dev = 'cuda:0'
CLIP_MODE = False
root_path = os.path.join(ROOT, model_dir)
path = os.path.join(root_path, "checkpoints/checkpoint_best.pt")
data_root = "/home/gridsan/lguan/keating/pacap/rfdiffusion/20240925/pacap_bind/outputs/wds" #
args_path = os.path.join(ROOT, model_dir, [u for u in os.listdir(os.path.join(ROOT, model_dir)) if u.endswith('.pt')][0])

backwards_compat = {
    'masked_rate': -1,
    'masked_mode': 'MASK',
    'lm_only_text': 1,
    'lm_weight': 1,
    'resid_weight': 1,
    'language_head': False,
    'language_head_type': 'MLP',
    'zip_enabled': False,
    'num_mutations': False,
}
hparams = torch.load(args_path)
args_dict = hparams['args']
args_dict['data_root'] = data_root
args_dict['batch_size'] = 1
args_dict['blacklist_file'] = ''
args_dict['num_workers'] = 1
for k in backwards_compat.keys():
    if k not in args_dict:
        args_dict[k] = backwards_compat[k]
args = SimpleNamespace(**args_dict)

print(vars(args))

coordinator_params = data_utils.get_coordinator_params(args.coordinator_hparams)
coordinator_params['num_positional_embeddings'] = args.gnn_num_pos_embs
coordinator_params['zero_out_pos_embs']= args.gnn_zero_out_pos_embs
coordinator_params['clip_mode'] = True

{'num_workers': 1, 'batch_size': 1, 'exp_name': '11_13_new_blacklist', 'epochs': 10, 'lr': 0.001, 'weight_decay': 0.001, 'momentum': 0.9, 'lr_peak_epoch': 2, 'label_smoothing': 0.0, 'disable_logging': 0, 'data_root': '/home/gridsan/lguan/keating/pacap/rfdiffusion/20240925/pacap_bind/outputs/wds', 'eval_epochs': 2, 'out_dir': 'runs/', 'only_non_bn_weight_decay': False, 'opt': 'ADAM', 'lr_scheduler': 'cosine', 'mixed_precision': 1, 'max_seq_len': 1024, 'self_supervised': 1, 'burn_in': 0, 'max_coord_len': 2000, 'freeze_llm': 0, 'freeze_text_proj': 0, 'finetune_from': '', 'num_mutations': -1, 'mutation_fwd_method': 'all', 'masked_rate': -1.0, 'masked_mode': 'MASK', 'lm_only_text': 1, 'lm_weight': 1.0, 'resid_weight': 1.0, 'zip_enabled': 0, 'zip_train_format_string': '', 'cath_info_dict': '', 'arch': 'facebook/esm2_t30_150M_UR50D', 'coordinator_hparams': 'terminator_configs/coordinator_broken_merge.json', 'gnn_checkpoint': '', 'gnn_num_pos_embs': 16, 'gnn_zero_out_pos_embs': False, 'languag

In [ ]:
## LOAD MODEL (NO CHANGES NEEDED)
args_dict['arch'] = '/data1/groups/keating_madry/huggingface/esm2_t30_150M_UR50D'
trained_model = model_utils.load_model(path, args_dict['arch'], dev)
tokenizer = EsmTokenizer.from_pretrained(args_dict['arch'])

esm_arch = args_dict['arch']
esm_model = EsmModel.from_pretrained(args_dict['arch']) 
esm_model = esm_model.to(dev)
esm_model.eval()
esm, alphabet = esmlib.pretrained.esm2_t30_150M_UR50D()
# esm, alphabet = esmlib.pretrained.esm1v_t33_650M_UR90S_1()
esm = esm.eval()
if dev == 'cuda:0':
    esm = esm.cuda()

loading state dict from /home/gridsan/lguan/keating/rla/model_weights/version_0/checkpoints/checkpoint_best.pt
building model based on path


Some weights of EsmModel were not initialized from the model checkpoint at /data1/groups/keating_madry/huggingface/esm2_t30_150M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ZERO OUT POS EMB False
GNN Potts Model Encoder hidden dimensionality is 128
freeze_llm False
use text proj:  True
args: 
320 True


Some weights of EsmModel were not initialized from the model checkpoint at /data1/groups/keating_madry/huggingface/esm2_t30_150M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
## LOAD UTIL FUNCTIONS (NO CHANGES NEEDED)
%run 2023_12_11_rla_utils.ipynb

/home/gridsan/lguan/miniconda3/envs/rla/lib/python3.9/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(
/state/partition1/slurm_tmp/27195077.0.0/ipykernel_1533856/2500981057.py:21: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  from scipy.stats.stats import pearsonr


In [ ]:
trained_model = trained_model.eval()

In [ ]:
## DEFINE RLA SETTING CALCULATIONS (SOME CHANGES POSSIBLE)
seq_mask='protein' # 'peptide' to mask peptide sequence, 'protein' to mask protein sequence
struct_mask=None # 'peptide' to mask peptide structure, 'protein' to mask protein structure
top_k = 30 # Num neighbors, probably want to keep at 30 but can experiment
focus = True # RLA calculation setting that limits RLA score to interface, almost certainly want to keep True
remove_far = True # Removes residues too far from the interface, likely want to keep True
threshold = 1 # Threshold for remove_far calculation, likely want to keep at 1
weight_dists = False # Weights RLA score per residue by distance from interface, likely want to keep False
pep_weight = 1 # Weight of peptide residues relative to protein residues, likely want to keep at 1
force_from_back = False # Define TRUE if masked sequence is always the last chain
force_from_front = True # Define TRUE if masked sequence is always the first chain

In [ ]:
## DATASETS TO EVALUATE (CHANGE AS NEEDED)
design_sets = ["data"]

In [ ]:
args.batch_size=1

In [ ]:
train_path = os.path.join(args.data_root, args.train_wds_path)
val_path = os.path.join(args.data_root, args.val_wds_path)
print('train_path:', train_path)
print('val_path:', val_path)

train_path: /home/gridsan/lguan/keating/pacap/rfdiffusion/20240925/pacap_bind/outputs/wds/train/shard-{000000..000019}.tar
val_path: /home/gridsan/lguan/keating/pacap/rfdiffusion/20240925/pacap_bind/outputs/wds/val/{000000..000004}.tar


In [12]:
## PERFORMS RLA CALCULATION (NO CHANGES NEEDED)
if CLIP_MODE:
    feature_getter = get_text_and_image_features_clip
else:
    feature_getter = get_text_and_image_features
    
torch.multiprocessing.set_sharing_strategy('file_system')
nclash_dict, Fnat_dict, Fnonnat_dict, LRMS_dict, iRMSDbb_dict, irmsdsc_dict, distance_dict, theta_dict, class_dict = {}, {}, {}, {}, {}, {}, {}, {}, {}
dicts = [nclash_dict, Fnat_dict, Fnonnat_dict, LRMS_dict, iRMSDbb_dict, irmsdsc_dict, distance_dict, theta_dict, class_dict]
nclash_data, Fnat_data, Fnonnat_data, LRMS_data, iRMSDbb_data, irmsdsc_data, distance_data, theta_data, class_data = {}, {}, {}, {}, {}, {}, {}, {}, {}
data_dicts = [nclash_data, Fnat_data, Fnonnat_data, LRMS_data, iRMSDbb_data, irmsdsc_data, distance_data, theta_data, class_data]
args.batch_size = 1
args.zip_enabled = False
args.num_mutations = 0
args.distributed = 0
plot_scores = []
plot_weights = []
plot_pep_mask = []
plot_indices = []
plot_X = []
plot_seq = []
paired_res_scores = {}
scores_stats = {'models': [], 'seqs': [], 'rla_scores': []}
# result_types = ['nclash', 'fnat', 'fnonnat', 'lrmsd', 'irmsdbb', 'irmsdsc', 'distance', 'theta', 'classification']
for design_set in design_sets:
    print(f'running on {design_set}.')
    args.train_wds_path = f"{design_set}.wds"
    args.val_wds_path = f"{design_set}.wds"
    print('train_path:', args.train_wds_path)
    print('val_path:', args.val_wds_path)
    train_loader, val_loader, train_len, val_len = get_wds_loaders(args, coordinator_params, gpu=None, shuffle_train=False, val_only=True, return_count=False)
    lens = {}
    for i, b in tqdm(enumerate(val_loader), total=val_len):
        lens[b[0]['pdb_id'][0]] = len(b[0]['string_sequence'][0])
    MAX_LEN = max(lens.values())
    
    for i, batch in enumerate(tqdm(val_loader, total=val_len)):
        model = batch[0]['pdb_id'][0]
        pep_seq = batch[0]['string_sequence'][0][:batch[1]['seq_lens'][0][0]]
        chain_lens = torch.zeros(batch[1]['coords'][0].shape[1]).to(device = batch[1]['coords'][0].device)
        chain_lens[batch[1]['seq_lens'][0][0]:] = 1
        chain_lens_mask = torch.ones(batch[1]['coords'][0].shape[1]).unsqueeze(0).to(dtype=torch.bool, device = batch[1]['coords'][0].device)
        batch[1]['chain_lens'] = [chain_lens.unsqueeze(0), chain_lens_mask]
        with torch.no_grad():
            with autocast(dtype=torch.float16):
                output_dict = feature_getter(trained_model, tokenizer, batch, 
                                             pdb=None, 
                                             weight_dists=weight_dists, 
                                             seq_mask=seq_mask, 
                                             focus=focus, 
                                             top_k=top_k, 
                                             struct_mask=struct_mask, 
                                             remove_far=remove_far, 
                                             threshold=threshold, 
                                             dev=dev,
                                             force_from_back=force_from_back,
                                             force_from_front=force_from_front
                                            )
                score, scores, plot_scores, plot_weights, plot_pep_mask, plot_indices, plot_X, plot_seq = compute_score(output_dict, weight_dists, MAX_LEN, plot_scores=plot_scores, plot_weights=plot_weights, plot_pep_mask=plot_pep_mask, plot_indices=plot_indices, plot_X=plot_X, plot_seq=plot_seq, is_complex=True)
                scores_stats['models'].append(model)
                scores_stats['seqs'].append(pep_seq)
                paired_res_scores[model] = score

running on data.
train_path: data.wds
val_path: data.wds
/home/gridsan/lguan/keating/pacap/rfdiffusion/20240925/pacap_bind/outputs/wds/data.wds False
{'max_coords_len': 2000, 'shuffle_coords': False, 'max_seq_len': 1024, 'pos_offset': 128, 'burn_in': 0, 'k_neighbors': 30, 'crop_type': 'absolute', 'shuffle_chains': False, 'num_mutations': 0, 'masked_rate': -1.0, 'masked_mode': 'MASK'}
added select filtering... 30
0 0


3000it [00:09, 308.86it/s]
3000it [03:20, 14.93it/s]


In [14]:
## SAVE/LOAD RAW SCORES AND SEQUENCE MAPPINGS (CHANGE JSON WRITING/READING AS NEEDED)
import json

with open('/home/gridsan/lguan/keating/pacap/rfdiffusion/20240925/pacap_bind/outputs/wds/seq_mapping.json', 'w') as f:
    json.dump(scores_stats, f)
with open('/home/gridsan/lguan/keating/pacap/rfdiffusion/20240925/pacap_bind/outputs/wds/scores.json', 'w') as f:
    json.dump(paired_res_scores, f)